In [37]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urlparse
import json

In [38]:
news = pd.read_csv("uci-news-aggregator.csv")

# utils

In [39]:
def remove_www(x):
    dot_split = x.split(".")
    if len(dot_split) > 2:
        return ".".join(dot_split[1:])
    else:
        return x

# cleaning list

In [40]:
news["HOSTNAME"] = news.HOSTNAME.apply(remove_www)

In [41]:
news = news.drop_duplicates(subset="HOSTNAME", keep="last")

In [42]:
news_domains = news.HOSTNAME.unique().tolist()

In [43]:
for i in news_domains:
    if ".edu" in i:
        news_domains.remove(i)
len(news_domains)

9549

In [44]:
for i in news_domains:
    if ".edu" in i:
        print(i)

hawaii.edu
uconn.edu
buffalo.edu
iwu.edu
uiowa.edu


In [45]:
for i in news_domains:
    if ".edu" in i:
        news_domains.remove(i)
len(news_domains)

9545

In [46]:
to_rem = [
    "youtube.com",
    "co.ke",
    "co.nz",
    "co.tt",
    "co.tz",
    "co.ug",
    "co.uk",
    "co.za",
    "com.au",
    "com.lb",
    "com.ng",
    "com.ph",
    "com.pk",
    "com.sg",
    "com.ua",
    "linkedin.com",
]

In [47]:
news_domains = [x for x in news_domains if x not in to_rem]
len(news_domains)

9529

In [48]:
to_add = ["techmeme.com"]
for i in to_add:
    news_domains.append(i)
len(news_domains)

9530

In [49]:
with open('news_domains.txt', 'w') as f:
    f.write(json.dumps(news_domains))